<div align="center" dir="auto">
<p dir="auto"><a href="https://colab.research.google.com/github/encord-team/encord-notebooks/blob/main/colab-notebooks/Micromodels-generate-detection-predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<div align="center" dir="auto">
  <div style="flex: 1; padding: 10px;">
    <a href="https://join.slack.com/t/encordactive/shared_invite/zt-1hc2vqur9-Fzj1EEAHoqu91sZ0CX0A7Q" target="_blank" style="text-decoration:none">
      <img alt="Join us on Slack" src="https://img.shields.io/badge/Join_Our_Community-4A154B?label=&logo=slack&logoColor=white">
    </a>
    <a href="https://docs.encord.com/docs/active-overview" target="_blank" style="text-decoration:none">
      <img alt="Documentation" src="https://img.shields.io/badge/docs-Online-blue">
    </a>
    <a href="https://twitter.com/encord_team" target="_blank" style="text-decoration:none">
      <img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/encord_team?label=%40encord_team&amp;style=social">
    </a>
    <img alt="Python versions" src="https://img.shields.io/pypi/pyversions/encord-active">
    <a href="https://pypi.org/project/encord-active/" target="_blank" style="text-decoration:none">
      <img alt="PyPi project" src="https://img.shields.io/pypi/v/encord-active">
    </a>
    <a href="https://docs.encord.com/docs/active-contributing" target="_blank" style="text-decoration:none">
      <img alt="PRs Welcome" src="https://img.shields.io/badge/PRs-Welcome-blue">
    </a>
    <img alt="Licence" src="https://img.shields.io/github/license/encord-team/encord-active">
  </div>
</div>

<div align="center">
  <p>
    <a align="center" href="" target="_blank">
      <img
        width="7232"
        src="https://storage.googleapis.com/encord-notebooks/encord_active_notebook_banner.png">
    </a>
  </p>
</div>

# 🟣 Encord Notebooks | Generate Encord Active Model Predictions Using Encord Micro-Models

#### Note-1: Ontologies of the Encord Active and Encord Annotate should be the same
#### Note-2: The data of the local Encord Active project should have been downloaded via the following CLI command:

```shell
encord-active project download-data
```


## Set parameters

In [1]:
ENCORD_ACTIVE_PROJECT_PATH='/path/to/local/encord_active/project'
ENCORD_SSH_KEY_PATH='ENCORD_SSH_KEY_PATH'
ENCORD_PROJECT_HASH='ENCORD_PROJECT_HASH'
ENCORD_MODEL_ITERATION_HASH='ENCORD_MODEL_ITERATION_HASH'

BATCH_SIZE=24
CONFIDENCE_THRESHOLD=0.6
IOU_THRESHOLD=0.3

## Import libraries

In [2]:
import os
import json
import pickle
from pathlib import Path

from encord.constants.enums import DataType

from encord import EncordUserClient, Project
from tqdm import tqdm
from encord_active.lib.common.data_utils import url_to_file_path
from encord_active.lib.db.connection import PrismaConnection
from encord_active.lib.db.predictions import (
    BoundingBox,
    Format,
    ObjectDetection,
    Prediction,
)
from encord_active.lib.project.project_file_structure import ProjectFileStructure

/opt/anaconda3/envs/ea_examples/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
user_client = EncordUserClient.create_with_ssh_private_key(Path(ENCORD_SSH_KEY_PATH).expanduser().read_text())
project: Project = user_client.get_project(ENCORD_PROJECT_HASH)

ea_project_fs: ProjectFileStructure = ProjectFileStructure(ENCORD_ACTIVE_PROJECT_PATH)
ontology = json.loads(ea_project_fs.ontology.read_text(encoding="utf-8"))
ontology_featureHashes = [obj["featureNodeHash"] for obj in ontology.get("objects")]

predictions_to_store = []
file_paths = []
data_hashes = []

with PrismaConnection(ea_project_fs) as conn:
    for label in tqdm(ea_project_fs.iter_labels(cache_db=conn), desc="Collecting images"):
        for du in label.iter_data_unit(cache_db=conn):
            file_path = url_to_file_path(du.signed_url, ea_project_fs.project_dir)
            if file_path is not None and du.data_type in [DataType.IMAGE.value, DataType.IMG_GROUP]:
                file_paths.append(file_path.as_posix())
                data_hashes.append(du.du_hash)

current_file_paths = []
current_data_hashes = []
pbar = tqdm(total=len(file_paths), desc="Running inference", leave=False)
for counter, (file_path, data_hash) in enumerate(zip(file_paths, data_hashes)):

    current_file_paths.append(file_path)
    current_data_hashes.append(data_hash)

    if (counter + 1) % BATCH_SIZE == 0 or counter + 1 == len(file_paths):
        try:
            inference_results = project.model_inference(
                ENCORD_MODEL_ITERATION_HASH,
                file_paths=current_file_paths,
                conf_thresh=CONFIDENCE_THRESHOLD,
                iou_thresh=IOU_THRESHOLD,
            )
        except Exception as e:
            print(e)
            continue

        for inference_result, current_data_hash in zip(inference_results, current_data_hashes):
            for obj in inference_result["predictions"]["0"]["objects"]:

                if obj["shape"] != "bounding_box":
                    print(f"prediction is not in `bbox` type (it is in {obj['shape']} type), skipping...")
                    continue

                if obj["featureHash"] not in ontology_featureHashes:
                    print(
                        f"'{obj['name']}' with featureHash '{obj['featureHash']}' is not available in the ontology of"
                        f" the Encord Active project."
                    )
                    continue

                prediction = Prediction(
                    data_hash=current_data_hash,
                    confidence=obj["confidence"],
                    object=ObjectDetection(
                        format=Format.BOUNDING_BOX,
                        data=BoundingBox(
                            x=obj["bounding_box"]["x"],
                            y=obj["bounding_box"]["y"],
                            w=obj["bounding_box"]["w"],
                            h=obj["bounding_box"]["h"],
                        ),
                        feature_hash=obj["featureHash"],
                    ),
                )
                predictions_to_store.append(prediction)

        current_file_paths = []
        current_data_hashes = []

    pbar.update(1)

prediction_file = f"predictions_{ENCORD_MODEL_ITERATION_HASH[:8]}.pkl"
with open((ea_project_fs.project_dir / prediction_file), "wb") as f:
    pickle.dump(predictions_to_store, f)

FileNotFoundError: [Errno 2] No such file or directory: 'ENCORD_SSH_KEY_PATH'

In [ ]:
os.chdir(ea_project_fs.project_dir.as_posix()) 
! encord-active import predictions {prediction_file}

print("\n✅︎ Predictions are imported successfully!")